In [1]:
import numpy as np
import pandas as pd


In [2]:
from dataset import load_dataset, load_labels, convert_to_epochs, load_channels
from features import time_series_features, fractal_features, entropy_features, hjorth_features, freq_band_features
from classifiers import KNN, SVM, NN
import variables as v

In [3]:
#Filtered from SAM40
dataset_ = load_dataset(data_type="filtered", test_type="Arithmetic")
dataset = convert_to_epochs(dataset_, v.NUM_CHANNELS, v.SFREQ)

#ICA filtered
dataset_ica_ = load_dataset(data_type="ica1", test_type="Arithmetic")
dataset_ica = convert_to_epochs(dataset_ica_, v.NUM_CHANNELS, v.SFREQ)

#ICA filtered two times
dataset_ica_2_ = load_dataset(data_type="ica2", test_type="Arithmetic")
dataset_ica_2 = convert_to_epochs(dataset_ica_2_, v.NUM_CHANNELS, v.SFREQ)

In [4]:
# time series features
features = time_series_features(dataset, v.NUM_CHANNELS)
#features_ica = time_series_features(dataset_ica, v.NUM_CHANNELS)
#features_ica_2 = time_series_features(dataset_ica_2, v.NUM_CHANNELS)

# hjorth features
#features = hjorth_features(dataset, v.NUM_CHANNELS, v.SFREQ)
features_ica = hjorth_features(dataset_ica, v.NUM_CHANNELS, v.SFREQ)
features_ica_2 = hjorth_features(dataset_ica_2, v.NUM_CHANNELS, v.SFREQ)

data = features
data_ica = features_ica
data_ica_2 = features_ica_2

In [5]:
labels = load_labels()
label = pd.concat([labels['t1_math'], labels['t2_math'],
                  labels['t3_math']]).to_numpy()
label = label.repeat(dataset.shape[1])

### KNN

In [6]:
print('Filtered data')
print(KNN(data,label))

Filtered data
(3000, 96)

 Confusion matrix:
[[288  23]
 [ 20 269]]
[0.92833333 0.93506494 0.92123288]


In [7]:
print('\nICA filtered data')
print(KNN(data_ica,label))



ICA filtered data
(3000, 64)

 Confusion matrix:
[[281  30]
 [ 41 248]]
[0.88166667 0.87267081 0.89208633]


In [8]:
print('\nICA filtered data round 2')
print(KNN(data_ica_2,label))


ICA filtered data round 2
(3000, 64)

 Confusion matrix:
[[303   8]
 [ 13 276]]
[0.965      0.95886076 0.97183099]


### SVM

In [9]:
print('Filtered data')
print(SVM(data,label))


Filtered data

 Confusion matrix:
[[297  14]
 [ 23 266]]
[0.93833333 0.928125   0.95      ]


In [15]:
print('\nICA filtered data')
print(SVM(data_ica,label))



ICA filtered data

 Confusion matrix:
[[284  27]
 [ 45 244]]
[0.88       0.86322188 0.900369  ]


In [11]:
print('\nICA filtered data round 2')
print(SVM(data_ica_2,label))


ICA filtered data round 2

 Confusion matrix:
[[299  12]
 [ 21 268]]
[0.945      0.934375   0.95714286]


# Neural Network

In [16]:
print('Filtered data')
NN(data,label)


Trial 1 Complete [00h 00m 22s]
val_accuracy: 0.8474999964237213

Best val_accuracy So Far: 0.8474999964237213
Total elapsed time: 00h 00m 22s

Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
4                 |3                 |layers
160               |736               |units_0
relu              |relu              |act_0
0.0001            |0.001             |learning_rate

Epoch 1/50
57/57 [==============================] - 1s 6ms/step - loss: 5.1141 - accuracy: 0.5128 - val_loss: 2.9345 - val_accuracy: 0.4700
Epoch 2/50
57/57 [==============================] - 0s 3ms/step - loss: 2.3277 - accuracy: 0.5044 - val_loss: 2.1446 - val_accuracy: 0.4783
Epoch 3/50
57/57 [==============================] - 0s 3ms/step - loss: 1.8147 - accuracy: 0.4983 - val_loss: 1.5956 - val_accuracy: 0.5183
Epoch 4/50
57/57 [==============================] - 0s 4ms/step - loss: 1.5124 - accuracy: 0.5217 - val_loss: 1.3689 - val_accuracy: 0.5600
Epoch 5/50
57/57 [============

FailedPreconditionError: {{function_node __wrapped__MergeV2Checkpoints_device_/job:localhost/replica:0/task:0/device:CPU:0}} Failed to rename: .\untitled_project\trial_1\checkpoint_temp/part-00000-of-00001.data-00000-of-00001 to: .\untitled_project\trial_1\checkpoint.data-00000-of-00001 : Prosessen får ikke tilgang til filen fordi den brukes av en annen prosess.
; Broken pipe [Op:MergeV2Checkpoints]

In [ ]:
print('\nICA filtered data')
NN(data_ica,label)


In [ ]:
print('\nICA filtered data round 2')
NN(data_ica_2,label)